The notebook presents the usage of new functions suggested to be included in the easygems package. The examples were prepared in the environment recommended for the global hackathon: https://github.com/digital-earths-global-hackathon/tools/blob/main/python_envs/environment.yaml

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import intake
from easygems import healpix as egh

import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Load ICON datasets from the global hackathon at 2 different zoom levels.

In [ ]:
cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")['online']

sim1_id = 'icon_d3hp003'
sim1_opts = {'time':'PT6H','time_method':'inst'}

ds1_zoom6  = cat[sim1_id](zoom=6,**sim1_opts).to_dask().pipe(egh.attach_coords)
ds1_zoom11 = cat[sim1_id](zoom=11,**sim1_opts).to_dask().pipe(egh.attach_coords)

Check approximate pixel size.

In [ ]:
egh.pix_size(egh.get_zoom(ds1_zoom6))

In [ ]:
egh.pix_size(egh.get_zoom(ds1_zoom11))

Select a subset from the coarse resolution dataset for the given time range and geographical domain.

In [ ]:
domain = np.array([-90, -80, -20, -10])
time_range = ('2020-08-01','2020-08-31')

rds1_zoom6 = egh.select_lonlat_rectangle(ds1_zoom6.sel(time=slice(*time_range)),domain)

From the fine resolution dataset, select the cells belonging to the cells of the coarse resolution selection. 

In [ ]:
chunksize = 4 ** (egh.get_zoom(ds1_zoom11)-egh.get_zoom(rds1_zoom6))
cells_fine = egh.isel_refine(rds1_zoom6.cell.values, chunksize)

rds1_zoom11 = ds1_zoom11.sel(time=rds1_zoom6.time).sel(cell=cells_fine)

Plot outgoing shortwave radiation for one timestep.

In [ ]:
def plot_map(da,**opts):
    fig, ax = plt.subplots(1,1,figsize=(4,4),
                           constrained_layout=True,
                           subplot_kw={"projection": ccrs.PlateCarree()})
    ax.set_extent(domain, crs=ccrs.PlateCarree())
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    im = egh.healpix_show(da, ax=ax, **opts)
    plt.colorbar(im, ax=ax, shrink=0.5, label=da.name)
    return fig, ax

In [ ]:
plot_time = np.datetime64('2020-08-01T18:00')

plot_map(rds1_zoom6['rsut'].sel(time=plot_time),cmap='Greys_r')
plot_map(rds1_zoom11['rsut'].sel(time=plot_time),cmap='Greys_r')

Check whether coarsening the high resolution dataset gives the same result.

In [ ]:
plot_map(
    egh.select_lonlat_rectangle(ds1_zoom11['rsut'].sel(time=plot_time),domain,zoom=6),
    cmap = 'Greys_r'
)

Now get the same geographical domain, time range at the same two zoom levels but for the IFS. The problem is there are only zoom 7 and 11 in the catalog. Therefore, to get regional dataset at zoom 6, load zoom 7 and use zoom parameter of the function select_lonlat_rectangle. The advantage is that the coarsening is executed for the region of interest only, not the whole globe.

In [ ]:
sim2_id = 'ifs_tco3999-ng5_rcbmf_cf'
sim2_opts = {'time':'PT1H'}

ds2_zoom7 = cat[sim2_id](zoom=7,**sim2_opts).to_dask().pipe(egh.attach_coords)

rds2_zoom6 = egh.select_lonlat_rectangle(
    ds2_zoom7.sel(time=rds1_zoom6.time,method='nearest'),
    domain,
    zoom=6
)

In the case of fine resolution, we can use the same cell indices as computed previously to select the proper region. This can take a moment.

In [ ]:
%%time

ds2_zoom11 = cat[sim2_id](zoom=11,**sim2_opts).to_dask().pipe(egh.attach_coords)

rds2_zoom11 = ds2_zoom11.sel(time=rds1_zoom6.time,method='nearest').sel(cell=cells_fine)

Plot one timestep.

In [ ]:
plot_map(rds2_zoom6['rsut'].sel(time=plot_time,method='nearest'),cmap='Greys_r')
plot_map(rds2_zoom11['rsut'].sel(time=plot_time,method='nearest'),cmap='Greys_r')

Again, check the result of coarsening the fine resolution.

In [ ]:
plot_map(
    egh.select_lonlat_rectangle(ds2_zoom11['rsut'].sel(time=plot_time,method='nearest'),domain,zoom=6),
    cmap = 'Greys_r'
)

The ICON instantaneous output does not contain cloud cover variable. Try computing it from total condensate qall - apply a threshold and project onto 2D. This should be done at highest available resolution and averaged to the desired one to get continous range of values between 0 and 1.

In [ ]:
%%time

def mask(da, threshold, dim=None):
    da = da>threshold
    if dim != None:
        da = da.sum(dim)>0
    return da

rds1_zoom6['clt'] = egh.coarsened_fun(
    rds1_zoom11['qall'],
    fun = mask,
    zoom_coarse = egh.get_zoom(rds1_zoom6),
    threshold = 1e-6,
    dim = 'pressure'
)

Plot a single timestep.

In [ ]:
plot_map(rds1_zoom6['clt'].sel(time=plot_time),cmap='Blues_r')